# CH 13 - Introduction to Modeling Libraries in Python

## 13.1 Interfacing Between pandas and Model Code

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y':  [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

* The `.values` attribute is intended to be used when your data is homogeneous—for example, all numeric types. If you have heterogeneous data, the result will be an ndarray of Python objects:

In [4]:
df3 = data.copy()

df3['strings'] = ['a', 'b', 'c', 'd', 'e']

df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [5]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

* For some models, you may only wish to use a subset of the columns. I recommend using `loc` indexing with `values`

In [6]:
model_cols = ['x0', 'x1']

data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

*  In Chapter 12 we looked at pandas’s `Categorical` type and the `pandas.get_dummies`
function. Suppose we had a non-numeric column in our example dataset

In [7]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [8]:
dummies = pd.get_dummies(data.category, prefix='category')

data_with_dummies = data.drop('category', axis=1).join(dummies)

data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 13.2 Creating Model Descriptions with Patsy

* Patsy is a Python library for describing statistical models (especially linear models) with a small string-based “formula syntax,” which is inspired by the formula syntax used by R programming languages.

* The ` patsy.dmatrices` function takes a formula string along with a dataset (which can be a DataFrame or a dict of arrays) and produces design matrices for a linear model.

In [9]:
data = pd.DataFrame({'x0': [1, 2, 3, 4, 5],
                     'x1': [0.01, -0.01, 0.25, -4.1, 0.],
                     'y':  [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [10]:
import patsy

In [11]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [12]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [13]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

#### These Patsy `DesignMatrix` instances are NumPy ndarrays with additional metadata

In [14]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [15]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

* You might wonder where the `Intercept` term came from. This is a convention for linear models like ordinary least squares (OLS) regression. You can suppress the intercept by adding the term `+ 0` to the model

In [16]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

* The Patsy objects can be passed directly into algorithms like `numpy.linalg.lstsq`, which performs an ordinary least squares regression:

In [17]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

<ipython-input-17-ef4372856a50>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid, _, _ = np.linalg.lstsq(X, y)


In [18]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [19]:
coef.squeeze()

array([ 0.31290976, -0.07910564, -0.26546384])

In [20]:
coef = pd.Series(coef.squeeze(),  index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

## 13.3 Introduction to statsmodels

* statsmodels is a Python library for fitting many kinds of statistical models, performing statistical tests, and data exploration and visualization. Statsmodels contains more “classical” frequentist statistical methods, while Bayesian methods and machine learning models are found in other libraries.

### Estimating Linear Models

* Linear models in statsmodels have two different main interfaces: array-based and formula-based. These are accessed through these API module imports

In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [23]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
            size=size,
    return mean + np.sqrt(variance) * np.random.randn(*size)
    
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0,.4, size=N),
          dnorm(0,.6, size=N),
          dnorm(0,.2, size=N)]

eps = dnorm(0, .1, size=N)
beta = [0.1, .3, .5]

y = np.dot(X, beta) + eps

* Here, I wrote down the “true” model with known parameters `beta`. In this case, `dnorm` is a helper function for generating normally distributed data with a particular mean and variance. So now we have:

In [25]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [26]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

* A linear model is generally fitted with an intercept term. The `sm.add_constant` function can add an intercept column to an existing matrix.

In [27]:
X_model = sm.add_constant(X)

X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

* The `sm.OLS` class can fit an ordinary least squares linear regression:

In [29]:
model = sm.OLS(y, X)

results = model.fit()

results.params

array([0.17826108, 0.22303962, 0.50095093])

In [30]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Sat, 08 Aug 2020   Prob (F-statistic):                    7.44e-12
Time:                        20:01:25   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Estimating Time Series Processes

* Another class of models in statsmodels are for time series analysis. Among these are autoregressive processes, Kalman filtering and other state space models, and multi-variate autoregressive models.

* Let’s simulate some time series data with an autoregressive structure and noise

In [32]:
init_x = 4

import random

values = [init_x, init_x]
N = 1000

b0 = .8
b1 = -.4
noise = dnorm(0, .1, N)

for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [33]:
values[:5]

[4, 4, 1.8977509636904242, 0.08686526220610424, -0.5769469132535335]

In [37]:
MAXLAGS = 5

model = sm.tsa.AR(values)

results = model.fit(MAXLAGS)

results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

In [38]:
print(results.summary())

                               AR Model Results                               
Dep. Variable:                      y   No. Observations:                 1002
Model:                          AR(5)   Log Likelihood                -241.974
Method:                          cmle   S.D. of innovations              0.308
Date:                Sat, 08 Aug 2020   AIC                             -2.338
Time:                        20:09:00   BIC                             -2.304
Sample:                             0   HQIC                            -2.325
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0062      0.010     -0.628      0.530      -0.025       0.013
L1.y           0.7845      0.032     24.696      0.000       0.722       0.847
L2.y          -0.4085      0.040    -10.108      0.0